---
title: 特征选择：方差分析
categories: statistics
date: 2020-12-30
---

在统计学里，方差分析（Analysis of Variance, ANOVA）一般用于检测不同组之间是否有显著性差异，它是由英国著名的统计学家是[Ronald Aylmer Fisher](https://baike.baidu.com/item/%E8%B4%B9%E5%B8%8C%E5%B0%94/8763421)首先提出。本文将会描述方差分析及其理论基础，然后描述机器学习中如何使用方差分析进行特征筛选。

![image-20201230101311660](images/image-20201230101311660.png)

## 方差分析

方差分析的理论基础是F分布，让我们首先来看一看。

 设$X\sim \chi^2(m),Y\sim\chi^2(n)$, 且$X, Y$相互独立，可以计算随机变量：
$$
Z=\frac{X/m}{Y/n}
$$
则，随机变量$Z$服从第一自由度为$m$、第二自由度为$n$的$F$分布，记$Z \sim F(m,n)$。其概率密度图如下。

![image-20201230113437584](images/image-20201230113437584.png)

有了F分布的理论，下面我们来看看它和方差分析是怎么建立关系的。

我们知道，在科学研究和商业分析中，经常要比较若干个因素对业务指标的影响。比如，为了比较药物A,B,C对治疗某疾病的疗效，将实验对象分成三组，分别记录服用三种药物的治疗效果，得到三组样本：
$$
y_{11},y_{12}, \dots,y_{1n_1}  \\
y_{21},y_{22}, \dots,y_{2n_2}  \\
y_{31},y_{32}, \dots,y_{3n_3}
$$
通过这些数据，希望回答：这三种药物对治疗该疾病有没有显著差异。首先我们假设这三种药物没有任何差异，也就说它们来自一个相同的正态分布。三组药物实验，可以看成从这个正态分布中分别做三组抽样。这样无疑满足了如下条件。

- 每个样本来自同一个正太分布。
- 三组样本的方差相同。

上诉条件满足了F分布的条件，由此可以构造如下统计量。明显可以看到，当​MSR​越小，F值越小，这表示这些样本更加可能来自于同一分布。

| 来源  | 自由度 | 平方和                                                   | 均方和             | F值                                 |
| ----- | ------ | -------------------------------------------------------- | ------------------ | ------------------------------------ |
| 因子 | $ r −1$ | $$SSR$$ | $MSR=\frac {SSR} {r-1}$ | $\frac {MSR} {MSE}$ |
| 误差  | $n−r$ | $SSE$   | $MSE=\frac {SSE} {n-r}$ |  |
| 总和  | $n−1$  | $SST$                                                   | $MST=\frac {SST} {n-1}$ |                                      |

- SSR（Sum of Squares Regression）：回归平方和，也称组件平方和。
  $$
  SSR = \sum_{i=1}^r n_i(\bar y_{i\cdot} -\bar y)^2
  $$

  - $r$：总共由$r$组样本。
  - $n_i$ ：第$i$组的样本个数。
  - $\bar y_{i\cdot}$ ：第$i$组的平均值
  - $\bar y$： 总体平均值。

- SSE（Sum of Squares Error）：残差平方和，也称组内平方和。
  $$
  SSE= \sum_{i=1}^r\sum_{j=1}^{n_i}(y_{ij}-\bar y_{i\cdot} )^2
  $$

  - $y_{ij}$：第$i$组的第$j$个样本值

- SST（Sum of Squares Total）：离差平方和，也称总平方和。
  $$
  SST = \sum_{i=1}^r\sum_{j=1}^{n_i}(y_{ij}-\bar y )^2
  $$
  不难证明如下公式。
  $$
  SST = SSR + SSE
  $$

- $R^2 $ ：当组间差异越大，说明不同组对样本的影响越大。可以用这个指标来衡量变量的关系强度。
  $$
  R^2 = \frac {SSR} {SST}
  $$

在机器学习分类任务，可以运用方差分析来筛选特征，以二分类（Positive和Negative）为例，可以把某一个特征的值分成两组，一组是Positive，一组是Negative，由此可以采用方差分析的方法。

## 实例分析

数据是来自于[1994年美国的人口普查数据](https://www.kaggle.com/uciml/adult-census-income?select=adult.csv)，其包含了人们的年龄，工作类型，教育，婚姻，人种，性别，每周工作时长，国家，收入等信息。我们的目标是根据这些信息预测收入（income），在建模之前，需要剔除找出那些和收入无关的特征。

### 数据

引入要使用的包。

In [3]:
import logging
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import six.moves.urllib as urllib
import zipfile

from IPython.display import display
from PIL import Image
from scipy import stats
from sklearn.feature_selection import SelectKBest, f_classif  
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.utils import check_array
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols

logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)

np.set_printoptions(suppress=True)

下载并加载数据。

In [4]:
def aduit_census_download(target_path, source_url="http://archive.ics.uci.edu/ml/machine-learning-databases/adult", http_proxy=None):
    if http_proxy is not None:
        proxy_handler = urllib.request.ProxyHandler({'http': http_proxy, 'https': http_proxy})
        opener = urllib.request.build_opener(proxy_handler)
    else:
        opener = urllib.request.build_opener()

    urllib.request.install_opener(opener)

    def maybe_download(file_name):
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        file_path = os.path.join(target_path, file_name)
        if not os.path.exists(file_path):
            source_file_url = os.path.join(source_url, file_name)
            logging.info(source_file_url)
            filepath, _ = urllib.request.urlretrieve(source_file_url, file_path)
            statinfo = os.stat(filepath)
            logging.info('Successfully downloaded {} {} bytes.'.format(file_name, statinfo.st_size))
        return file_path
    
    csv_file = 'adult.data'
    data_path= maybe_download(csv_file)
    
    extract_path = os.path.join(target_path, csv_file)
    return extract_path

local_path = os.path.join('.', 'data/adult_census')
data_path = aduit_census_download(local_path)

cols = ['age', 'workclass', 'fnlwg', 'education', 'education-num', 
       'marital-status','occupation','relationship', 'race','sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
df_census = pd.read_csv(data_path, names=cols, sep=', ', engine='python')
df_census.head(5)

2021-03-31 16:05:07,082: INFO: http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
2021-03-31 16:05:18,734: INFO: Successfully downloaded adult.data 3974305 bytes.


,age,workclass,fnlwg,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


![image-20201230142618554](images/image-20201230142618554.png)

收入有两个值，一个是`<=50K`，一个是`>50K`。

In [5]:
list(df_census.income.unique())

['<=50K', '>50K']

![image-20201230150113588](images/image-20201230150113588.png)

### 连续特征选取

根据前面的定义，可知方差分析的方法只适用于选择连续特征。下面使用[sklearn.feature_selection.SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)对这些特征进行特征选取。其中最后一列是随机数，我们希望采用方差分析的方法剔除这个列。

In [6]:
# 添加一个随机数列，下面将验证这个列的卡方统计值
np.random.seed(1229)
df_census['random'] = np.random.randint(0, 10, (len(df_census),1))

continuous_columns = ['age', 'fnlwg', 'education-num', 'capital-gain', 
                      'capital-loss', 'hours-per-week', 'random']

X = df_census[continuous_columns].to_numpy()
y = df_census[['income']].to_numpy().ravel()

print('-'*25, 'X', '-'*25)
print(X[0:5], X.shape)

print('-'*25, 'X_selected', '-'*25)
select_kbest = SelectKBest(f_classif, k=3)
X_selected = select_kbest.fit_transform(X, y)

print(X_selected[0:5], X_selected.shape)

print('-'*25, 'KBest scores', '-'*25)
print('KBest scores=', np.round(select_kbest.scores_, 0))
print('KBest pvalues=', np.round(select_kbest.pvalues_, 4)) 

------------------------- X -------------------------
[[    39  77516     13   2174      0     40      0]
 [    50  83311     13      0      0     13      7]
 [    38 215646      9      0      0     40      5]
 [    53 234721      7      0      0     40      9]
 [    28 338409     13      0      0     40      3]] (32561, 7)
------------------------- X_selected -------------------------
[[39 13 40]
 [50 13 13]
 [38  9 40]
 [53  7 40]
 [28 13 40]] (32561, 3)
------------------------- KBest scores -------------------------
KBest scores= [1887.    3. 4120. 1709.  755. 1813.    2.]
KBest pvalues= [0.     0.0877 0.     0.     0.     0.     0.2198]


![image-20201230144039809](images/image-20201230144039809.png)

上面的结果可以看到，`fnlwg`列和最后一列（随机列）的F统计量最小，而且p值超过0.05，这样我们可以接受这两列和收入之间的关系是独立的，也就是说，可以剔除这两列。上面代码中函数`f_classif`实现了方差分析的功能，为了验证，我们可以手工计算一遍。

In [7]:
def split(X, y):
            
    # 根据y对X进行分组    
    index = y.argsort()
    X = X[index]
    y = y[index]
            
    groups = np.split(X, np.unique(y, return_index=True)[1][1:])     
    return groups
        
def anova_score(groups, significant=0.05):   
    def square_of_sums(a, axis=0):
        s = np.sum(a, axis=0)
        return s.astype(float) * s

    def sum_of_squares(a, axis=0):
        return np.sum(a*a, axis=axis)     
    
    sse = np.sum([sum_of_squares(g) - square_of_sums(g)/len(g) for g in groups])
    a = np.concatenate(groups)
    n = len(a)
    l = len(groups)
    s = np.sum(a)
    sst = sum_of_squares(a) - square_of_sums(a)/n
    ssa = sst - sse
    mse = sse/(n-l)
    msa = ssa/(l-1)
    f = msa/mse
    p_value = 1-stats.f.cdf(f, l-1, n-l)
    return f, l-1, n-1, p_value
       
scores = [ anova_score(split(X[:,j], y)) for j in range(X.shape[1])] 
print(scores)
print('-'*25, 'scores', '-'*25)
print('scores=', [round(f, 0) for f, _, _, _ in scores])
print('p_values=', [round(p_value, 4) for _, _, _, p_value in scores]) 

[(1886.7073137161203, 1, 32560, 1.1102230246251565e-16), (2.9155935856443214, 1, 32560, 0.08773666108297618), (4120.095779707474, 1, 32560, 1.1102230246251565e-16), (1709.1500637437975, 1, 32560, 1.1102230246251565e-16), (754.8304515515131, 1, 32560, 1.1102230246251565e-16), (1813.386282216147, 1, 32560, 1.1102230246251565e-16), (1.5056046325401369, 1, 32560, 0.21981987970702077)]
------------------------- scores -------------------------
scores= [1887.0, 3.0, 4120.0, 1709.0, 755.0, 1813.0, 2.0]
p_values= [0.0, 0.0877, 0.0, 0.0, 0.0, 0.0, 0.2198]


![image-20201230144141445](images/image-20201230144141445.png)

可以看到scores和p_values和SelectKBest的结果完全相同。除了上面两种方法，我们还可以采用[statsmodels.stats.anova.anova_lm](https://www.statsmodels.org/stable/generated/statsmodels.stats.anova.anova_lm.html)实现相同的效果。

In [8]:
df_census.columns = [name.replace('-', '_') for name in df_census.columns]
continuous_columns = [name.replace('-', '_') for name in continuous_columns]

for column in continuous_columns:
    formula = '{}~C(income)'.format(column)
    print('-'*25, formula, '-'*25)
    lm = ols('{}~C(income)'.format(column) , data=df_census).fit()
    display(anova_lm(lm))

------------------------- age~C(income) -------------------------


,df,sum_sq,mean_sq,F,PR(>F)
C(income),1.0,3.318258e+05,331825.767179,1886.707314,0.0
Residual,32559.0,5.726333e+06,175.875593,NaN,NaN


------------------------- fnlwg~C(income) -------------------------


,df,sum_sq,mean_sq,F,PR(>F)
C(income),1.0,3.248013e+10,3.248013e+10,2.915594,0.087737
Residual,32559.0,3.627119e+14,1.114014e+10,NaN,NaN


------------------------- education_num~C(income) -------------------------


...

,df,sum_sq,mean_sq,F,PR(>F)
C(income),1.0,1.197936e+08,1.197936e+08,754.830452,2.686547e-164
Residual,32559.0,5.167200e+09,1.587026e+05,NaN,NaN


------------------------- hours_per_week~C(income) -------------------------


,df,sum_sq,mean_sq,F,PR(>F)
C(income),1.0,2.618895e+05,261889.502853,1813.386282,0.0
Residual,32559.0,4.702175e+06,144.420141,NaN,NaN


------------------------- random~C(income) -------------------------


,df,sum_sq,mean_sq,F,PR(>F)
C(income),1.0,12.316628,12.316628,1.505605,0.21982
Residual,32559.0,266349.525883,8.180519,NaN,NaN


![image-20201230160907946](images/image-20201230160907946.png)

![image-20201230160926087](images/image-20201230160926087.png)

## 参考

- [方差分析](https://wh136.github.io/anova/anova.pdf)
- [如何理解方差分析和F分布？](https://blog.csdn.net/ccnt_2012/article/details/109815989)
- [如何理解t检验、t分布、t值？](https://www.matongxue.com/madocs/580/)
- [数理统计讲义](https://bookdown.org/hezhijian/book/)
- [scipy.stats.f_oneway](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html)
- [统计学原理实验教程（Python）](https://github.com/SharkFin-top/Statistics_Python_Codes/)

## 历史

- 2020-12-30：初始版本。

In [75]:
class MultiSelectKBest(object):
    def __init__(self, score_func=f_classif, k=10):
        super().__init__()
        self.score_func = score_func
        self.k = k
        self.pvalues_ = None
        self.scores_ = None

    def fit(self, X, y):
        scores = []
        pvalues = []
        for j in range(y.shape[1]):
            selector = SelectKBest(self.score_func, k='all')
            selector.fit(X, y[:, j])
            scores.append(list(selector.scores_))
            if selector.pvalues_ is not None:
                pvalues.append(list(selector.pvalues_))

        self.scores_ = np.mean(scores, axis=0)
        if len(pvalues) > 0:
            self.pvalues_ = np.mean(pvalues, axis=0)
        else:
            self.pvalues_ = None

        return self
        
    def transform(self, X):
        if self.k=='all':
            return X
        scores = self.scores_
        indexes = np.argsort(scores, kind="mergesort")[-self.k:]
        indexes.sort()
        return X[:, indexes]
        
    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X)

In [79]:
new_y = df_census[['income', 'sex']].to_numpy()
new_y

array([['<=50K', 'Male'],
       ['<=50K', 'Male'],
       ['<=50K', 'Male'],
       ...,
       ['<=50K', 'Female'],
       ['<=50K', 'Male'],
       ['>50K', 'Female']], dtype=object)

In [81]:
select_kbest = MultiSelectKBest(f_classif, k=3)
X_selected = select_kbest.fit_transform(X, new_y)
print(select_kbest.scores_)
print(select_kbest.pvalues_)
print(X_selected[0:5])

[1072.83820875   13.20956613 2062.50320439  892.92648914  411.28804903
 1810.22300183    0.77943825]
[0.         0.04386896 0.0133494  0.         0.         0.
 0.51864305]
[[39 13 40]
 [50 13 13]
 [38  9 40]
 [53  7 40]
 [28 13 40]]


In [87]:
select_kbest1.scores_ 

array([1886.70731372,    2.91559359, 4120.09577971, 1709.15006374,
        754.83045155, 1813.38628222,    1.50560463])

In [88]:
select_kbest2.scores_ 

array([ 258.96910378,   23.50353867,    4.91062908,   76.70291454,
         67.74564651, 1807.05972145,    0.05327186])

In [90]:
(select_kbest1.scores_  + select_kbest2.scores_ )/2 

array([1072.83820875,   13.20956613, 2062.50320439,  892.92648914,
        411.28804903, 1810.22300183,    0.77943825])

In [84]:
select_kbest1 = SelectKBest(f_classif, k=3)
y1 = df_census[['income']].to_numpy().ravel()
X_selected = select_kbest1.fit_transform(X, y1)
print(select_kbest1.scores_)
print(select_kbest1.pvalues_)
print(X_selected[0:5])

[1886.70731372    2.91559359 4120.09577971 1709.15006374  754.83045155
 1813.38628222    1.50560463]
[0.         0.08773666 0.         0.         0.         0.
 0.21981988]
[[39 13 40]
 [50 13 13]
 [38  9 40]
 [53  7 40]
 [28 13 40]]


In [86]:
select_kbest2 = SelectKBest(f_classif, k=3)
y1 = df_census[['sex']].to_numpy().ravel()
X_selected = select_kbest2.fit_transform(X, y1)
print(select_kbest2.scores_)
print(select_kbest2.pvalues_)
print(X_selected[0:5])

[ 258.96910378   23.50353867    4.91062908   76.70291454   67.74564651
 1807.05972145    0.05327186]
[0.         0.00000125 0.02669881 0.         0.         0.
 0.81746621]
[[  39 2174   40]
 [  50    0   13]
 [  38    0   40]
 [  53    0   40]
 [  28    0   40]]


In [64]:
print(X[0:1])
print(X_selected[0:5])

[[   39 77516    13  2174     0    40     0]]
[[39 13 40]
 [50 13 13]
 [38  9 40]
 [53  7 40]
 [28 13 40]]


In [23]:
y

array(['<=50K', '<=50K', '<=50K', ..., '<=50K', '<=50K', '>50K'],
      dtype=object)